## HW4 -Solutions

Columbia Uni:yg2499

Please build a predictive model that can classify a Company Name into any of these industries. 

Submit the URL to the your github repo that contains the assignment

1.  Pre-processing engine to read in 100 random companies per industry.  Under-represented industries you need to over-sample (with replacement) and over-represented you need to under-sample (without replacement) - 40 points

2.  Data Cleansing - 20 points

3.  Transform data and Train a model - 20 points

4.   Classify arbitrary text for specific industry - 20 points

Code can run continuously without need to save off pickle files

Rescources: [Get Bad Accuracy of classify](https://stackoverflow.com/questions/45070186/machine-learning-to-classify-company-names-to-their-industries)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
#from sklearn.cross_validation import StratifiedShuffleSplit,StratifiedKFold,train_test_split
from scipy import stats
from sklearn import decomposition,linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn.manifold import Isomap
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier,Lasso,SGDClassifier,LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,f1_score,hamming_loss
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score,precision_recall_curve,f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.manifold import Isomap
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error
import re
from nltk.corpus import stopwords,wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import SGDClassifier

In [2]:
# load the dataset
thepath ='/Users/gyang/Desktop/ProjectDataScience/HW/HW4/'

cdata_0 =pd.read_csv(thepath +'private us companies.csv')

/Users/gyang/anaconda/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cdata_0.head()

,Unnamed: 0,Company,Industry,Unnamed: 3
0,NaN,A - E Employees Credit Union,Diversified Financial Services,
1,NaN,"A & A Contract Services, Inc.",Professional Services,
2,NaN,"A & A Express, Inc.",Road and Rail,
3,NaN,"A & A Fertilizer, Ltd.",Chemicals,
4,NaN,A & A Food Service. Inc.,Distributors,


In [4]:
cdata_1 =cdata_0[['Company','Industry']]

In [5]:
cdata_1.head()

,Company,Industry
0,A - E Employees Credit Union,Diversified Financial Services
1,"A & A Contract Services, Inc.",Professional Services
2,"A & A Express, Inc.",Road and Rail
3,"A & A Fertilizer, Ltd.",Chemicals
4,A & A Food Service. Inc.,Distributors


In [6]:
cdata_1.shape

(327684, 2)

In [7]:
cdata_1.Company.nunique()

325927

In [8]:
cdata_1.Industry.nunique()

99

In [9]:
#cdata_1['Industry'].value_counts()

cdata_1.iloc[:,1] = cdata_1.iloc[:,1].map(lambda x: re.sub(r'[^a-zA-Z]+', ' ',x)) #remove special chars
cdata_1.iloc[:,1] = cdata_1.iloc[:,1].map(str.strip) #remove strip

/Users/gyang/anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


### Data Pre-processing - 40 points

Pre-processing engine to read in 100 random companies per industry.  
Under-represented industries you need to over-sample (with replacement) and over-represented you need to under-sample (without replacement) - 40 points

In [54]:
size = 100      # sample size
replace = True  # with replacement
fn = lambda x: x.loc[np.random.choice(x.index, size, replace),:]
cdata_2 =cdata_1.groupby('Industry', as_index=False).apply(fn)

In [55]:
cdata_2.head()

Company               Industry
0 129504  Harris Acoustic Products Corporation  Aerospace and Defense
  261237                         Skandia, Inc.  Aerospace and Defense
  153735           Jet Parts Engineering, Inc.  Aerospace and Defense
  116341            GE Power System Composites  Aerospace and Defense
  829                 AAA Aircraft Supply, LLC  Aerospace and Defense

In [56]:
cdata_2.shape

(9900, 2)

In [57]:
cdata_2.Industry.nunique()

99

In [58]:
cdata_2.Company.nunique()

8597

In [14]:
#mydict = dict(zip(cdata_2.Company, cdata_2.Industry))
#len(mydict)

### Data Cleansing - 20 points

In [15]:
cdata_2.iloc[:,0] = cdata_2.iloc[:,0].map(lambda x: re.sub(r'[^a-zA-Z]+', ' ',x)) #remove special chars
cdata_2.iloc[:,0] = cdata_2.iloc[:,0].map(str.strip) #remove strip
cdata_2.iloc[:,0] = cdata_2.iloc[:,0].apply(lambda x: x.lower()) #lower case
cdata_2.iloc[:,0] = cdata_2.iloc[:,0].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ') #remove single letter

In [16]:
cdata_2.head()

Company               Industry
0 155757                        jrm technologies inc  Aerospace and Defense
  36558   bizjet international sales and support inc  Aerospace and Defense
  189587               miniature machine corporation  Aerospace and Defense
  21288                        archangel systems inc  Aerospace and Defense
  32540                                    belac llc  Aerospace and Defense

In [17]:
cdata_3 =cdata_2.copy(deep=True)

In [18]:
cdata_3.head()

Company               Industry
0 155757                        jrm technologies inc  Aerospace and Defense
  36558   bizjet international sales and support inc  Aerospace and Defense
  189587               miniature machine corporation  Aerospace and Defense
  21288                        archangel systems inc  Aerospace and Defense
  32540                                    belac llc  Aerospace and Defense

### Transform data and Train a model - 20 points

In [19]:
cdata_4 =pd.DataFrame({'Words':cdata_3.groupby('Industry')['Company'].apply(','.join)}).reset_index()
#cdata_4['Words'] = cdata_4['Words'].astype('object')

In [20]:
cdata_4.head()

,Industry,Words
0,Aerospace and Defense,"jrm technologies inc,bizjet international sale..."
1,Air Freight,"associated global systems inc,atlas freighter ..."
2,Air Freight and Logistics,"brick mountain logistics llc,brown integrated ..."
3,Airlines,"new vectors aviation inc,cj systems aviation g..."
4,Appliances and Furniture,"aquion inc,apadana inc,art connection inc,auro..."


In [21]:
cdata_4.shape

(99, 2)

In [22]:
cdata_4['Words'] = cdata_4['Words'].map(str.strip)
#cdata_4['Words'] = cdata_4['Words'].str.strip() #remove whitespaces
cdata_4['Words'] = cdata_4['Words'].str.replace(',', ' ') #remove comma from each string
cdata_4['Words'] = cdata_4['Words'].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ') #remove single letter
cdata_4['Words'] = cdata_4['Words'].map(str.strip)

In [23]:
#remove stopwords
stop =['inc','group','llc','corporation','lp','ltd','corp','incorporated',
       'company','services','and','for','the','holdings','at','co']
for word in stop:
    cdata_4['Words'] = cdata_4['Words'].str.replace(word, '')

In [24]:
cdata_4.head()

,Industry,Words
0,Aerospace and Defense,jrm technologies bizjet internional sales su...
1,Air Freight,associed global systems las freighter leasing...
2,Air Freight and Logistics,brick mountain logistics brown integred logis...
3,Airlines,new vectors aviion cj systems aviion metro ...
4,Appliances and Furniture,aquion apadana art nnection aurora brs apa...


In [27]:
train_text =list(cdata_4['Words'])

In [28]:
#label encode
le = preprocessing.LabelEncoder()
cdata_4['code'] = le.fit_transform(cdata_4['Industry'])
cdata_4['code'] = cdata_4['code'].astype('category')

In [29]:
train_target =cdata_4['code'].values

In [30]:
train_target

[0, 1, 2, 3, 4, ..., 94, 95, 96, 97, 98]
Length: 99
Categories (99, int64): [0, 1, 2, 3, ..., 95, 96, 97, 98]

In [31]:
text_clfPipe = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('feat-sel',  SelectFromModel(LinearSVC())),
                      ('clf-rf', MultinomialNB()),
])

In [32]:
text_clf = text_clfPipe.fit(train_text,train_target )

### Classify arbitrary text for specific industry - 20 points

In [34]:
testdata =cdata_1.copy(deep=True)

In [35]:
testdata.head()

,Company,Industry
0,A - E Employees Credit Union,Diversified Financial Services
1,"A & A Contract Services, Inc.",Professional Services
2,"A & A Express, Inc.",Road and Rail
3,"A & A Fertilizer, Ltd.",Chemicals
4,A & A Food Service. Inc.,Distributors


In [36]:
testdata.iloc[:,0] = testdata.iloc[:,0].map(lambda x: re.sub(r'[^a-zA-Z]+', ' ',x)) #remove special chars
testdata.iloc[:,0] = testdata.iloc[:,0].map(str.strip) #remove strip
testdata.iloc[:,0] = testdata.iloc[:,0].apply(lambda x: x.lower()) #lower case
testdata.iloc[:,0] = testdata.iloc[:,0].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ') #remove single letter

In [37]:
#remove stopwords
stop =['inc','group','llc','corporation','lp','ltd','corp','incorporated',
       'company','services','and','for','the','holdings','at','co']
for word in stop:
    testdata['Company'] = testdata['Company'].str.replace(word, '')

In [38]:
testdata.head()

,Company,Industry
0,employees credit union,Diversified Financial Services
1,ntract,Professional Services
2,express,Road and Rail
3,fertilizer,Chemicals
4,food service,Distributors


In [39]:
le = preprocessing.LabelEncoder()
testdata['code'] = le.fit_transform(testdata['Industry'])

In [40]:
testdata.shape

(327684, 3)

In [41]:
testText =list(testdata['Company'])

In [42]:
predicted = text_clf.predict(testText)

In [43]:
predicted

array([28, 48, 76, ...,  0,  0, 15])

In [44]:
accuracy_score(testdata['code'].values, predicted)

0.20712027441071276

In [45]:
from sklearn import metrics
print(metrics.classification_report(testdata['code'].values, predicted))

             precision    recall  f1-score   support

          0       0.01      0.36      0.03      3082
          1       0.00      0.41      0.01        46
          2       0.29      0.28      0.29      1302
          3       0.15      0.64      0.24       371
          4       0.02      0.39      0.04        71
          5       0.07      0.04      0.05      2205
          6       0.02      0.30      0.04        90
          7       0.04      0.27      0.08       554
          8       0.81      0.72      0.77      8392
          9       0.01      0.19      0.03       145
         10       0.41      0.41      0.41      1467
         11       0.28      0.11      0.16      2183
         12       0.12      0.08      0.10      2537
         13       0.04      0.28      0.07       185
         14       0.51      0.30      0.38     10452
         15       0.52      0.17      0.26      5615
         16       0.06      0.34      0.11       119
         17       0.31      0.08      0.13   

In [46]:
list(le.inverse_transform(predicted))

['Diversified Financial Services',
 'Household Durables',
 'Railroads and Trucking Services',
 'Food Beverages',
 'Food and Staples Retailing',
 'Trading Companies and Distributors',
 'Industrial Conglomerates',
 'Real Estate',
 'Machinery',
 'Construction and Engineering',
 'Metals and Mining',
 'Construction Materials',
 'Multiline Retail',
 'Building Products',
 'Aerospace and Defense',
 'Engineering and Construction Services',
 'Commercial Services and Supplies',
 'Electronic Equipment Instruments and Components',
 'Railroads and Trucking Services',
 'Chemicals',
 'Food and Staples Retailing',
 'Food Products',
 'Chemicals',
 'Engineering and Construction Services',
 'Conglomerates',
 'Construction and Engineering',
 'Construction and Engineering',
 'Construction and Engineering',
 'Diversified Services',
 'Aerospace and Defense',
 'Sea Transportation',
 'Machinery',
 'Multiline Retail',
 'Diversified Consumer Services',
 'Railroads and Trucking Services',
 'Trading Companies and D

Conclusion: 
    
The accuracy is lower than I expect because company's name is too short to predict.One way is to extract company's description one by one(We have 327684 company list,so word corpus will huge).Another way is to build word2vec ,and use Recurrent Neural Network(RNN/LSTM) to build deep learning algorthim to improve accuracy.